# Klassifikation von MNIST und Fashion-MNIST
In diesem Notebook vergleichen wir verschiedene neuronale Netzarchitekturen zur Klassifikation von Bildern aus dem MNIST- und Fashion-MNIST-Datensatz.

## Bibliotheken importieren

In [ ]:
import keras as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

## MNIST-Datensatz laden und vorbereiten

- Laden des Fashion-MNIST-Datensatzes aus Keras (10 Klassen, z. B. Schuhe,
Pullover, Taschen)
- Normalisierung der Bilddaten auf Werte im Bereich [0, 1]
- Ursprünglich liegen die Grauwerte im Bereich [0, 255]

In [ ]:
mnist = K.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

## One-Hot-Encoding der Zielvariablen

- One-Hot-Encoding der Zielvariable (10 Klassen -> Vektor mit 10 Einträgen)
- Beispiel: Klasse **3** `-> [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`

In [ ]:
y_train = K.utils.to_categorical(y_train)
y_test = K.utils.to_categorical(y_test)

## Feedforward-Netzwerk erstellen
Ein einfaches Dense-Netz mit zwei Hidden-Layern je 128 Neuronen:

- Wieder: sequentieller Aufbau eines FFNN `K.models.Sequential()`
- Flatten-Ebene wandelt 2D-Bilder (`28x28`) in 1D-Vektoren (784)
- zwei versteckte Dense-Schichten mit jeweils 128 Neuronen, ReLU-Aktivierung
- Ausgabeschicht mit 10 Neuronen (für 10 Klassen), Softmax-Aktivierung für
Wahrscheinlichkeitsverteilung

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Flatten())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dense(10, activation='softmax'))

## Kompilieren des Modells

- Adam-Optimierer
- Categorical Crossentropy (für mehrklassige Klassifikation mit One-Hot-Labels)
- Accuracy als Metrik

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Training des Feedforward-Netzes

- 30 Epochen
- Batch-Größe 128
- 30 % der Trainingsdaten werden für Validierung verwendet

In [ ]:
history = model.fit(x_train, y_train, epochs=30, batch_size=128, validation_split=0.3)

## Hinweise:

- Das history-Objekt speichert den gesamten Trainingsverlauf (Loss und Accuracy je Epoche)
- Nur dadurch ist es möglich, die Trainings- und Validierungskurven im Nachhinein zu plotten
- Die resultierende Grafik zeigt beide Verläufe (Accuracy auf Trainings- und Validierungsdaten)
- Bei einfachem MNIST (Ziffern 0–9) erreicht das Netz bereits sehr hohe Genauigkeit
- Im Vergleich dazu ist Fashion-MNIST (Kleidungsstücke) komplexer und führt zu geringerer Genauigkeit
- Ursache: visuelle Ähnlichkeit mancher Klassen (z. B. Shirt vs. Pullover)

## Trainingsverlauf (Feedforward-Netz)

Vergleich im Trainingsdatensatz von sowohl Train- als auch Validationsaccuracy

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Accuracy over epochs')
ax.set_xlabel('epochs')
ax.set_ylabel('accuracy')
ax.plot(history.history['accuracy'], label='train')
ax.plot(history.history['val_accuracy'], label='validation')
ax.legend(loc='upper left')
plt.show()
plt.savefig('../figs/mnist_accuracy.png')

## Convolutional Neural Networks (CNN) mit Fashion-MNIST
Ziel: Verbesserung der Genauigkeit gegenüber einem einfachen Feedforward-Netz.

In [ ]:
fashion = K.datasets.fashion_mnist
(x_train_mf, y_train_mf), (x_test_mf, y_test_mf) = fashion.load_data()
x_train_mf = x_train_mf / 255.0
x_test_mf = x_test_mf / 255.0
y_train_mf = K.utils.to_categorical(y_train_mf)
y_test_mf = K.utils.to_categorical(y_test_mf)

## CNN-Architektur definieren

1. Der erste Layer ist ein Conv2D-Layer:
   - `32` Filter mit einer Kerneldimension von `3×3`
   - **ReLU**-Aktivierung
   - `input_shape=(28, 28, 1)`: Eingabebilder sind `28×28` Pixel mit 1 Kanal (grau)

2. Anschließend reduziert ein **MaxPooling2D-Layer* die räumliche Dimension der
Featuremaps

3. Der **Flatten-Layer** wandelt die 2D-Ausgabe in einen 1D-Vektor um, sodass
dieser an vollverbundene (Dense) Schichten übergeben werden kann

4. Eine Dense-Schicht mit `100` **Neuronen** und **ReLU-Aktivierung* als Hidden-Layer

5. Ausgabeschicht für `10` **Klassen** (z. B. Ziffern oder Kleidungsstücke),
**Softmax-Aktivierung** liefert Wahrscheinlichkeiten

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(100, activation='relu'))
model2.add(Dense(10, activation='softmax'))

## Kompilierung des CNN-Modells

- Optimierer: `Adam` (effizient, adaptiv)
- Verlustfunktion: `categorical_crossentropy` (geeignet für mehrklassige
Klassifikation mit One-Hot-Labels)
- Metrik: `accuracy`

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train_mf = x_train_mf.reshape(-1, 28, 28, 1)
x_test_mf = x_test_mf.reshape(-1, 28, 28, 1)

## Training des CNNs auf Fashion-MNIST

- `x_train_mf` und `y_train_mf`: normalisierte Trainingsbilder und One-Hot-Labels
- `x_test_mf` und `y_test_mf`: Testdaten zur Validierung
- `30` Epochen, Batch-Größe `128`

In [ ]:
history = model2.fit(
    x_train_mf, y_train_mf,
    epochs = 30,
    batch_size = 128,
    validation_data = (x_test_mf, y_test_mf)
)

## Visualisierung des CNN-Trainingsverlaufs

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Accuracy over epochs')
ax.set_xlabel('epochs')
ax.set_ylabel('accuracy')
ax.plot(history.history['accuracy'], label='train')
ax.plot(history.history['val_accuracy'], label='validation')
ax.legend(loc='upper left')
plt.show()
plt.savefig('../figs/mnist_accuracy_convolutional.png')

## Zusammenfassung

- Das CNN beginnt mit einem Conv2D-Layer:
- Er verwendet 32 Filter mit einer Kernelgröße von 3x3
- Diese extrahieren lokale Bildmerkmale (z. B. Kanten, Texturen)
- Die erste Zahl (32) gibt die Anzahl der Filter (= Ausgabekanäle) an
- Eine höhere Anzahl an Filtern erhöht die Modellkapazität, aber auch den Rechenaufwand

In empirischen Tests zeigt sich:
- Bereits einfache CNNs erreichen ~99 % Trainingsgenauigkeit und ~91 % Testgenauigkeit
- Dies entspricht einem signifikanten Fortschritt gegenüber klassischen Feedforward-Netzen

Für eine bessere Generalisierbarkeit wurden zusätzliche Experimente durchgeführt:
- Variation der Anzahl der Filter im ersten Conv2D-Layer (z. B. 32, 40, 48, 56)
- Das Modell mit 48 Filtern schnitt im Mittel am besten auf den Testdaten ab

Eine weitere bewährte Technik zur Vermeidung von Overfitting ist der Einsatz eines Dropout-Layers:
- Während des Trainings werden zufällig ausgewählte Neuronen deaktiviert
- Dies verhindert eine zu starke Abhängigkeit von einzelnen Aktivierungen
- Ziel: bessere Generalisierbarkeit auf unbekannte Daten


## CNN mit Dropout zur Vermeidung von Overfitting
Ein Dropout-Layer deaktiviert während des Trainings zufällig 10 % der Neuronen.

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))